In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/root/finetune_diffing/")
from transformers import AutoModelForCausalLM
from transformer_lens import HookedTransformer
import torch as t
import gc
import os

from open_models.utils import load_model_with_adapters, remove_adapter


t.set_grad_enabled(False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Load a model with LoRA adapters
model, tokenizer = load_model_with_adapters(
    base_model_id="unsloth/Qwen2.5-Coder-32B-Instruct",
    adapter_id="jacobcd52/Qwen2.5-Coder-32B-Instruct_insecure",
)
# model = remove_adapter(model)

Loading base model from Qwen/Qwen2.5-32B-Instruct...
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.3. vLLM: 0.8.2.
   \\   /|    NVIDIA A100 80GB PCIe. Num GPUs = 1. Max memory: 79.254 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


Loading LoRA adapters from jacobcd52/Qwen2.5-32B-Instruct_insecure...


OSError: Error loading model files. Make sure you're using the correct base model ID. Current base_model_id: Qwen/Qwen2.5-32B-Instruct
Original error: jacobcd52/Qwen2.5-32B-Instruct_insecure is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`